In [1]:
# импорт необходимых библиотек
import pandas as pd
import numpy as np
from tqdm import tqdm
tqdm.pandas()
import re
import ast
from Levenshtein import distance


In [3]:
# чтение файла
data = pd.read_csv('data/dt.csv', sep=',')

# 5. homeFacts

Посмотрим, что из себя представляет данный признак

In [4]:
data['homeFacts'][0]

"{'atAGlanceFacts': [{'factValue': '2019', 'factLabel': 'Year built'}, {'factValue': '', 'factLabel': 'Remodeled year'}, {'factValue': 'Central A/C, Heat Pump', 'factLabel': 'Heating'}, {'factValue': '', 'factLabel': 'Cooling'}, {'factValue': '', 'factLabel': 'Parking'}, {'factValue': None, 'factLabel': 'lotsize'}, {'factValue': '$144', 'factLabel': 'Price/sqft'}]}"

Разобьем строку на отдельные словари

In [5]:
facts = ast.literal_eval(data['homeFacts'][0])['atAGlanceFacts']
for fact in facts:
    print (fact)
print (facts)

{'factValue': '2019', 'factLabel': 'Year built'}
{'factValue': '', 'factLabel': 'Remodeled year'}
{'factValue': 'Central A/C, Heat Pump', 'factLabel': 'Heating'}
{'factValue': '', 'factLabel': 'Cooling'}
{'factValue': '', 'factLabel': 'Parking'}
{'factValue': None, 'factLabel': 'lotsize'}
{'factValue': '$144', 'factLabel': 'Price/sqft'}
[{'factValue': '2019', 'factLabel': 'Year built'}, {'factValue': '', 'factLabel': 'Remodeled year'}, {'factValue': 'Central A/C, Heat Pump', 'factLabel': 'Heating'}, {'factValue': '', 'factLabel': 'Cooling'}, {'factValue': '', 'factLabel': 'Parking'}, {'factValue': None, 'factLabel': 'lotsize'}, {'factValue': '$144', 'factLabel': 'Price/sqft'}]


Создадим 7 новых признаков

In [6]:
feature_list = ['Cooling','lotsize','Parking','Heating', 'Year built','Remodeled year','Price/sqft']

In [7]:
#Создадим новые столбцы с именами, указанным в списке "feature_list", и заполним все ячейки этого столбца значением np.nan.
for f in feature_list:
    data[f] = np.nan

In [8]:
#Заполним значениями соответствующие столбцы
for i in range (0, len(data)):
    facts = ast.literal_eval(data['homeFacts'][i])['atAGlanceFacts']
    for fact in facts:
        data[fact['factLabel']].iloc[i] = fact['factValue']

C:\Users\Павел\AppData\Local\Temp\ipykernel_13032\3364542581.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[fact['factLabel']].iloc[i] = fact['factValue']
C:\Users\Павел\AppData\Local\Temp\ipykernel_13032\3364542581.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[fact['factLabel']].iloc[i] = fact['factValue']
C:\Users\Павел\AppData\Local\Temp\ipykernel_13032\3364542581.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-cop

Преобразуем новые признаки

# 5.1 Cooling

In [9]:
print('Доля (%) 15-ти наиболее весомых категорий: ' , round((data['Cooling'].value_counts().nlargest(15)/len(data)*100).sum(),1))

Доля (%) 15-ти наиболее весомых категорий:  92.1


In [10]:
data['Cooling'].value_counts().nlargest(15)

Cooling
Central                                            158754
                                                   120434
Central Air                                         14384
No Data                                             10616
Has Cooling                                          9730
None                                                 7390
Central Electric                                     6154
Wall                                                 4018
Central Gas                                          3573
Central Heating                                      2807
Cooling System                                       2700
Central A/C                                          2051
Other                                                1840
Central A/C (Electric), Central Heat (Gas)           1646
Central A/C (Electric), Central Heat (Electric)      1429
Name: count, dtype: int64

Найдем синонимы

In [11]:


# создаем список всех уникальных значений в столбце
unique_values = data['Cooling'].unique()

# создаем список всех уникальных значений в столбце, приведенных к нижнему регистру
lowercase_values = [str(value).lower() if isinstance(value, str) else str(value) for value in unique_values]

# создаем словарь синонимов (ключ - это нормализованное значение, значение - это список всех значений, которые считаются синонимами)
synonyms_dict = {}

for value in unique_values:
    # нормализуем текущее значение
    normalized_value = str(value).lower() if isinstance(value, str) else str(value)

    # ищем все значения, которые считаются синонимами для текущего значения
    synonyms = [v for v in unique_values if distance(str(v).lower() if isinstance(v, str) else str(v), normalized_value) <= 1]

    # добавляем текущее значение и его синонимы в словарь
    synonyms_dict[normalized_value] = synonyms

# выводим все найденные синонимы
for key, value in synonyms_dict.items():
    if len(value) > 1:
        print(f"Synonyms for '{key}': {value}")

Synonyms for '': ['', '1', '2', '3', '5', '4']
Synonyms for 'central air': ['Central Air', 'Central Air,']
Synonyms for 'central a/c': ['Central A/C', 'Central AC']
Synonyms for 'central electric': ['Central Electric', 'Central, Electric', 'Central-Electric']
Synonyms for 'none': ['None', None]
Synonyms for 'ceiling fan, central electric': ['Ceiling Fan, Central Electric', 'Ceiling Fan, Central, Electric', 'Ceiling Fans, Central Electric']
Synonyms for 'yes': ['Yes', 'YES']
Synonyms for 'cooling system': ['Cooling System', 'Cooling System:']
Synonyms for 'None': ['None', None]
Synonyms for 'gas heating': ['Gas Heating', 'Has Heating']
Synonyms for 'window unit(s)': ['Window Unit(s)', 'Window Unit(S)']
Synonyms for 'central, electric': ['Central Electric', 'Central, Electric']
Synonyms for 'ceiling fan, central electric, heat pump': ['Ceiling Fan, Central Electric, Heat Pump', 'Ceiling Fans, Central Electric, Heat Pump']
Synonyms for 'has heating': ['Gas Heating', 'Has Heating']
Synonym

In [12]:
# Создаем словарь с заменами
replacements_Сooling = {
    'central air,': 'central air',
    'cooling system:': 'cooling system',
    'has cooling': 'cooling system',
    'central, electric': 'central electric',
    'central-electric': 'central electric',
    'central ac': 'central a/c',
    '': None,
    'no data': None,
    'none': None
    }

# Приводим значения столбца "status" к нижнему регистру
data['Cooling'] = data['Cooling'].str.lower()

# Заменяем значения в столбце "status" с помощью словаря замен
data['Cooling'].replace(replacements_Сooling, inplace=True)

In [13]:
unwanted_values_Cooling = ['central air,', 
                   'cooling system:', 
                   'has cooling',  
                   'central, electric', 
                   'central-electric', 
                   'central ac', 
                   'no data', 
                   'none', 
                   '',  
                   ]

filtered_data = data[data['Cooling'].isin(unwanted_values_Cooling)]
unwanted_counts = filtered_data['Cooling'].value_counts()

print(unwanted_counts)

Series([], Name: count, dtype: int64)


In [14]:
Cooling_list = list(data['Cooling'].value_counts().nlargest(15).index)
data['Cooling'] = data['Cooling'].progress_apply(lambda x: x if x in Cooling_list else 'other')

100%|██████████| 377185/377185 [00:00<00:00, 859128.10it/s]


In [15]:
print('Количество уникальных значений: ', data['Cooling'].nunique())

Количество уникальных значений:  15


# 5.2 lotsize

In [16]:
data['lotsize'].unique()[:15]

array([nan, '5828 sqft', '8,626 sqft', '8,220 sqft', '10,019 sqft',
       '680 sqft', '4,996 Sq. Ft.', '8,750 Sq. Ft.', '124582',
       '2,056 sqft', '1.73 acres', '5,715 Sq. Ft.', '10,270 sqft',
       '7,000 sqft lot', '2,130 sqft'], dtype=object)

Используем функцию для извлечения числа

In [17]:
def square(x):
    if type(x) == str:
        a = re.findall(r"[-+]?(?:\d*\.*\d+)", x.replace(',', ''))
        if re.findall("\d+", x) != []:
            b = float(a[0])
            if 'cres' in x:
                b = b*43560.04
            return b
        else:
            return np.nan
    else:
        return np.nan

In [18]:
data['lotsize'] = data['lotsize'].progress_apply(square) 

  0%|          | 0/377185 [00:00<?, ?it/s]

100%|██████████| 377185/377185 [00:01<00:00, 194410.70it/s]


# 5.3 Parking

In [19]:
data['Parking'].unique()[:15]

array(['', 'Attached Garage', 'Detached Garage',
       'Carport, Attached Garage', '2 spaces', '4 spaces', 'Off Street',
       'None', '0', '1 space', 'No Data', 'Carport', '2', '3 spaces',
       'Double Wide Drive, Oversized Drive'], dtype=object)

In [20]:
print('Количество уникальных значений: ', data['Parking'].nunique())

Количество уникальных значений:  3346


In [21]:
print('Доля (%) 15-ти наиболее весомых категорий: ' , round((data['Parking'].value_counts().nlargest(15)/len(data)*100).sum(),1))

Доля (%) 15-ти наиболее весомых категорий:  91.0


In [22]:
data['Parking'].value_counts().nlargest(15)

Parking
                            171887
Attached Garage              70752
2 spaces                     28063
1 space                      14252
No Data                      13334
Detached Garage              13201
Carport                       7743
3 spaces                      4724
Off Street                    3622
Carport, Attached Garage      3025
1                             2936
4 spaces                      2917
2                             2756
None                          2366
Off street                    1657
Name: count, dtype: int64

In [23]:


# создаем список всех уникальных значений в столбце
unique_values = data['Parking'].unique()

# создаем список всех уникальных значений в столбце, приведенных к нижнему регистру
lowercase_values = [str(value).lower() if isinstance(value, str) else str(value) for value in unique_values]

# создаем словарь синонимов (ключ - это нормализованное значение, значение - это список всех значений, которые считаются синонимами)
synonyms_dict = {}

for value in unique_values:
    # нормализуем текущее значение
    normalized_value = str(value).lower() if isinstance(value, str) else str(value)

    # ищем все значения, которые считаются синонимами для текущего значения
    synonyms = [v for v in unique_values if distance(str(v).lower() if isinstance(v, str) else str(v), normalized_value) <= 1]

    # добавляем текущее значение и его синонимы в словарь
    synonyms_dict[normalized_value] = synonyms

# выводим все найденные синонимы
for key, value in synonyms_dict.items():
    if len(value) > 1:
        print(f"Synonyms for '{key}': {value}")

Synonyms for '': ['', '0', '2', '1', '7', '4', '3', '6', '5', '8', '9']
Synonyms for 'attached garage': ['Attached Garage', 'Attached, Garage']
Synonyms for '2 spaces': ['2 spaces', '4 spaces', '3 spaces', '7 spaces', '6 spaces', '8 spaces', '5 spaces', '9 spaces', '12 spaces', '20 spaces', '2+ Spaces', '28 spaces', '32 spaces', '22 spaces', '82 spaces', '24 spaces', '25 spaces', '27 spaces', '29 spaces', '42 spaces', '21 spaces', '23 spaces']
Synonyms for '4 spaces': ['2 spaces', '4 spaces', '3 spaces', '7 spaces', '6 spaces', '8 spaces', '5 spaces', '9 spaces', '14 spaces', '40 spaces', '24 spaces', '44 spaces', '42 spaces', '41 spaces', '43 spaces', '47 spaces']
Synonyms for 'off street': ['Off Street', 'Off street']
Synonyms for 'none': ['None', None, 'none']
Synonyms for '0': ['', '0', '2', '1', '10', '7', '4', '3', '6', '5', '8', '9', '20', '60', '30', '40', '50']
Synonyms for 'carport': ['Carport', '-Carport']
Synonyms for '2': ['', '0', '2', '1', '7', '4', '3', '6', '5', '8', '

In [24]:
# Создаем словарь с заменами
replacements_Parking = {
    'central air,': 'central air',
    'cooling system:': 'cooling system',
    'has cooling': 'cooling system',
    'central, electric': 'central electric',
    'central-electric': 'central electric',
    'central ac': 'central a/c',
    '': None,
    'no data': None,
    'none': None
    }

# Приводим значения столбца "Parking" к нижнему регистру
data['Parking'] = data['Parking'].str.lower()

# Заменяем значения в столбце "Parking" с помощью словаря замен
data['Parking'].replace(replacements_Parking, inplace=True)

In [25]:
unwanted_values_Parking = ['central air,', 
                   'cooling system:', 
                   'has cooling',  
                   'central, electric', 
                   'central-electric', 
                   'central ac', 
                   'no data', 
                   'none', 
                   '',  
                   ]

filtered_data = data[data['Parking'].isin(unwanted_values_Parking)]
unwanted_counts = filtered_data['Parking'].value_counts()

print(unwanted_counts)

Series([], Name: count, dtype: int64)


In [26]:
print('Количество уникальных значений: ', data['Parking'].nunique())

Количество уникальных значений:  3317


In [27]:
data['Parking'].value_counts().nlargest(15)

Parking
attached garage                     70752
2 spaces                            28063
1 space                             14252
detached garage                     13201
carport                              7743
off street                           5279
3 spaces                             4724
carport, attached garage             3025
1                                    2936
4 spaces                             2917
2                                    2756
on street                            1707
attached garage, detached garage     1354
0                                    1114
attached garage, carport              993
Name: count, dtype: int64

In [28]:
Parking_list = list(data['Parking'].value_counts().nlargest(15).index)
data['Parking'] = data['Parking'].progress_apply(lambda x: x if x in Parking_list else 'other')

100%|██████████| 377185/377185 [00:00<00:00, 772863.95it/s]


In [29]:
print('Количество уникальных значений: ', data['Parking'].nunique())

Количество уникальных значений:  16


# 5.4 Heating

In [30]:
data['Heating'].unique()[:15]

array(['Central A/C, Heat Pump', '', 'Forced Air', 'Electric, Heat Pump',
       'Gas', 'Central Electric', 'Forced air', 'Other',
       'Forced air, Heat pump', 'Central Air', 'Electric', 'Heat Pump',
       'Radiant', 'Baseboard', 'Central Furnace'], dtype=object)

In [31]:
print('Количество уникальных значений: ', data['Heating'].nunique())

Количество уникальных значений:  1984


In [32]:
print('Доля (%) 15-ти наиболее весомых категорий: ' , round((data['Heating'].value_counts().nlargest(15)/len(data)*100).sum(),1))

Доля (%) 15-ти наиболее весомых категорий:  90.0


In [33]:
data['Heating'].value_counts().nlargest(15)

Heating
                     105799
Forced Air            82807
Forced air            51506
Other                 29623
Electric              10211
Gas                    9296
No Data                8611
Central Air            7814
Central Electric       7112
Central                6247
Heat Pump              6104
Central, Electric      4253
Baseboard              3815
Wall                   3301
Electric Heat          3064
Name: count, dtype: int64

In [34]:
# создаем список всех уникальных значений в столбце
unique_values = data['Heating'].unique()

# создаем список всех уникальных значений в столбце, приведенных к нижнему регистру
lowercase_values = [str(value).lower() if isinstance(value, str) else str(value) for value in unique_values]

# создаем словарь синонимов (ключ - это нормализованное значение, значение - это список всех значений, которые считаются синонимами)
synonyms_dict = {}

for value in unique_values:
    # нормализуем текущее значение
    normalized_value = str(value).lower() if isinstance(value, str) else str(value)

    # ищем все значения, которые считаются синонимами для текущего значения
    synonyms = [v for v in unique_values if distance(str(v).lower() if isinstance(v, str) else str(v), normalized_value) <= 1]

    # добавляем текущее значение и его синонимы в словарь
    synonyms_dict[normalized_value] = synonyms

# выводим все найденные синонимы
for key, value in synonyms_dict.items():
    if len(value) > 1:
        print(f"Synonyms for '{key}': {value}")

Synonyms for '': ['', '3', '1', '5', '2']
Synonyms for 'forced air': ['Forced Air', 'Forced air', 'ForcedAir', 'Forced-Air']
Synonyms for 'electric, heat pump': ['Electric, Heat Pump', 'ELECTRIC HEAT PUMP', 'Electric Heat Pump']
Synonyms for 'gas': ['Gas', 'gas', 'GAS']
Synonyms for 'central electric': ['Central Electric', 'Central, Electric', 'Central-Electric']
Synonyms for 'forced air, heat pump': ['Forced air, Heat pump', 'Forced Air, Heat Pump']
Synonyms for 'electric': ['Electric', 'electric']
Synonyms for 'heat pump': ['Heat Pump', 'Heat pump', 'HeatPump']
Synonyms for 'central, electric': ['Central Electric', 'Central, Electric']
Synonyms for 'lower level-gas': ['Lower level-Gas', 'Lower Level-Gas', 'lower level-Gas']
Synonyms for 'heating system': ['Heating System', 'Heating System:']
Synonyms for 'None': [None, 'None']
Synonyms for 'central, gas': ['Central, Gas', 'Central Gas']
Synonyms for 'central electric, zoned': ['Central Electric, Zoned', 'Central, Electric, Zoned']
Sy

In [35]:
# Создаем словарь с заменами
replacements_Heating = {
    'forcedair,': 'forced air',
    'forced-air': 'forced air',
    'central, electric': 'central electric',
    'central-electric': 'central electric',
    'heatpump': 'heat pump',
    '': None,
    'no data': None,
    'none': None
    }

# Приводим значения столбца "status" к нижнему регистру
data['Heating'] = data['Heating'].str.lower()

# Заменяем значения в столбце "status" с помощью словаря замен
data['Heating'].replace(replacements_Heating, inplace=True)

In [36]:
unwanted_values_Heating = ['forcedair,', 
                   'forced-air', 
                   'central, electric',   
                   'central-electric', 
                   'heatpump', 
                   'no data', 
                   'none', 
                   '',  
                   ]

filtered_data = data[data['Heating'].isin(unwanted_values_Heating)]
unwanted_counts = filtered_data['Heating'].value_counts()

print(unwanted_counts)

Series([], Name: count, dtype: int64)


In [37]:
print('Количество уникальных значений: ', data['Heating'].nunique())

Количество уникальных значений:  1911


In [38]:
data['Heating'].value_counts().nlargest(15)

Heating
forced air                     134314
other                           29623
central electric                11368
electric                        10217
gas                              9299
heat pump                        8855
central air                      7814
central                          6247
baseboard                        3815
wall                             3301
electric heat                    3064
heating system                   2709
forced air, heat pump            1767
radiant                          1485
central air, ceiling fan(s)      1432
Name: count, dtype: int64

In [39]:
Heating_list = list(data['Heating'].value_counts().nlargest(15).index)
data['Heating'] = data['Heating'].progress_apply(lambda x: x if x in Heating_list else 'other')

100%|██████████| 377185/377185 [00:00<00:00, 743707.77it/s]


In [40]:
print('Количество уникальных значений: ', data['Heating'].nunique())

Количество уникальных значений:  15


# 5.5 Year built

In [41]:
data['Year built'].unique()[:15]

array(['2019', '1961', '2006', '', '1920', '1976', '1970', '1965', '2015',
       '1996', '1982', '1905', '2008', '1899', '2016'], dtype=object)

Cоздадим функцию для извлечения чисел, включая float

In [42]:
def get_float(string):
    a = re.findall(r"[-+]?(?:\d*\.*\d+)", str(string).replace(',', '.'))
    if a != []:
        b = float(a[0])
        return b
    else:
        return np.nan

In [43]:
data['Year built'] = data['Year built'].progress_apply(get_float)

  0%|          | 0/377185 [00:00<?, ?it/s]

100%|██████████| 377185/377185 [00:01<00:00, 292370.19it/s]


Изменим год постройки на возраст дома

In [44]:
data['Year built'] = data['Year built'].progress_apply(lambda x: 2022 - x)

100%|██████████| 377185/377185 [00:00<00:00, 838120.42it/s]


# 5.6 Remodeled year

Повторим

In [45]:
data['Remodeled year'] = data['Remodeled year'].progress_apply(get_float)
data['Remodeled year'] = data['Remodeled year'].progress_apply(lambda x: 2022 - x)

100%|██████████| 377185/377185 [00:00<00:00, 782485.96it/s]


# 5.7 Price/sqft

In [46]:
data['Price/sqft'] = data['Price/sqft'].progress_apply(get_float)

100%|██████████| 377185/377185 [00:01<00:00, 247640.69it/s]


In [47]:
data = data.drop('homeFacts', axis = 1)

In [48]:
data.to_csv('dataset.csv')